In [ ]:
!python cornell_demo.py --flor 

I would like to query the KVS above:
1. Get the loss in a plotting format
2. Get the accuracy in a plotting format
3. Compose (You are now building a dashboard)

In [ ]:
import flor

In [ ]:
base_df = flor.load_kvs()
base_df

In [ ]:
SELECT = ['vid']
FROM = base_df
WHERE = (base_df['projid'] == 'cornelldbg') 

q = FROM.loc[WHERE, SELECT]
q

In [ ]:
q['vid'].drop_duplicates().to_list()

In [ ]:
from sh import tail
import json

In [ ]:
ds = []
for path in q['vid'].drop_duplicates().to_list():
    eof = tail('-1', path, _iter=True).next()
    ds.append(json.loads(eof))
ds

In [ ]:
! tail -1 '/home/ubuntu/.flor/cornelldbg/2022-01-31T14:11:24.json'

In [ ]:
vid_mapper = {'/home/ubuntu/.flor/cornelldbg/2022-01-31T14:11:24.json': "a05e564bcd4caca42f4c50588537f5b3b84e8470"}

In [ ]:
q['vid'] = q['vid'].apply(lambda x: vid_mapper[x])

In [ ]:
q['vid']

### Now I want to see some tensorboard logging
1. log the loss and show in dashboard // or matplotlib your own
2. hindsight log error points and backpropagate.


Let's first get error points

In [ ]:
!ls

In [ ]:
flor.flags.NAME

In [ ]:
flor.flags.INDEX

In [ ]:
k = '1.loss'

In [ ]:
int(k.split('.')[0])

In [ ]:
'.'.join(k.split('.')[1:])

In [ ]:
!cat .replay.json